In [1]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, modified_precision
from nltk.metrics import scores
import scipy.io.wavfile
from IPython.display import Audio
from IPython.display import display

import seaborn as sns
%matplotlib inline

In [2]:
from nmt_run import *

In [3]:
model_files = [f for f in os.listdir(os.path.dirname(model_fil))
                   if os.path.basename(model_fil).replace('.model','') in f]
# print(model_files)

NameError: name 'model_fil' is not defined

In [ ]:
max_model_fil = max(model_files, key=lambda s: int(s.split('_')[-1].split('.')[0]))

In [ ]:
max_model_fil

In [ ]:
last_epoch = check_model()

In [ ]:
import nltk.translate.bleu_score

### Fisher

In [ ]:
key='fisher_train'
train=False
m_dict = map_dict[key]
v_dict = vocab_dict[dec_key]
n=len(map_dict[key])
BATCH_SIZE = 100

In [ ]:
list(m_dict.keys())[:5]

In [ ]:
en_words = []
_ = [en_words.extend(w['en_w']) for w in m_dict.values()]

es_words = []
_ = [es_words.extend(w['es_w']) for w in m_dict.values()]

In [ ]:
len(en_words), len(set(en_words)), len(es_words), len(set(es_words))

In [ ]:
def create_vocab(word_list, prune_freq=0):
    vocab = {}
    for w in word_list:
        if w in vocab:
            vocab[w] += 1
        else:
            vocab[w] = 1
    # end for
    
    # prune 
    vocab = {k:v for k,v in vocab.items() if v >= prune_freq}
    return vocab

In [ ]:
en_vocab = create_vocab(en_words, prune_freq=1)
es_vocab = create_vocab(es_words)

In [ ]:
en_words_pruned = [w for w in en_words if w in en_vocab]

In [ ]:
np.max([len(w) for w in en_vocab])

In [ ]:
len_range = (np.min([len(w) for w in en_vocab]), np.max([len(w) for w in en_vocab]))

In [ ]:
len_range

In [ ]:
x = np.array([len(w) for w in en_words_pruned])
len_counts = {i:0 for i in range(1,np.max(x)+1,1)}

for i in x:
    len_counts[i] += 1

In [ ]:
plt.figure(figsize=(12,5))
_ = sns.barplot(x=list(len_counts.keys()), y = list(len_counts.values()))

In [ ]:
# os.chdir(".."k,v)
# os.chdir("chainer2/speech2text")
os.chdir("/afs/inf.ed.ac.uk/group/project/lowres/work/chainer2/speech2text")

In [ ]:
en_words_by_len = {i: set([w for w in en_vocab if len(w) ==  i]) for i in range(1,np.max(x)+1,1)}
en_word_counts_by_len = {i: [w for w in en_words_pruned if len(w) ==  i] for i in range(1,np.max(x)+1,1)}

In [ ]:
start_len = 12
total_words_above_len = 0
total_word_counts_above_len = 0
for i in range(start_len,np.max(x)+1,1):
    total_words_above_len += len(en_words_by_len[i])
    total_word_counts_above_len += len(en_word_counts_by_len[i])
    print("{0:5d} | {1:5d} | {2:5d} | {3:5d}".format(i, 
                                                     len(en_words_by_len[i]), 
                                                     total_words_above_len,
                                                     total_word_counts_above_len))

print("\ntotal english words={0:d}".format(len(en_words_pruned)))

# start_len = 15
# total_word_counts_above_len = 0
# for i in range(start_len,np.max(x)+1,1):
#     total_word_counts_above_len += len(en_word_counts_by_len[i])
#     print("{0:5d} | {1:5d} | {2:5d}".format(i, len(en_words_by_len[i]), total_word_counts_above_len))
# print(total_word_counts_above_len)



In [ ]:
en_words_above_len = {k:v for k,v in en_vocab.items() if len(k) >= start_len}

In [ ]:
max([(w,len(w)) for w in en_words_above_len], key=lambda t:t[1])

In [ ]:
# from nltk.stem import *
# from nltk.stem.porter import *
# from nltk.stem.snowball import SnowballStemmer

In [ ]:
# stemmer = PorterStemmer()
# stemmer = SnowballStemmer("english")
# lemmer = nltk.wordnet.WordNetLemmatizer()
from stemming.porter2 import stem
import stemming

In [ ]:
stems_lems = [(w.decode("utf-8"), stem(w.decode("utf-8")), stem(w.decode("utf-8"))) for w in en_words_above_len]

In [ ]:
stem('residencies'), stem('residencies')

In [ ]:
en_word_stems = [stem(w.decode()) for w in en_vocab]

In [ ]:
len(set(en_vocab.keys())), len(set(en_word_stems))

In [ ]:
en_word_stem_freq = {}
for w in en_vocab:
    if stem(w.decode()) not in en_word_stem_freq:
        en_word_stem_freq[stem(w.decode())] = [w.decode()]
    else:
        en_word_stem_freq[stem(w.decode())].append(w.decode())

In [ ]:
en_word_stem_cats = sorted(en_word_stem_freq.items(), reverse=True, key=lambda t: len(t[1]))

In [ ]:
display_pp = PrettyTable(["stem","words"], hrules=True)
display_pp.align = "l"
for stem, word_list in en_word_stem_cats:
    if len(word_list) > 1:
        display_pp.add_row([stem, textwrap.fill(", ".join(word_list),50)])

print(display_pp)

In [ ]:
x = np.array([len(v) for k,v in en_word_stem_cats if len(v)>1])
len_counts = {i:0 for i in range(1,np.max(x)+1,1)}

for i in x:
    len_counts[i] += 1

In [ ]:
plt.figure(figsize=(12,8))
_ = sns.barplot(x=list(len_counts.keys()), y = list(len_counts.values()), palette="Blues_d")

In [ ]:
sorted(stems_lems)

In [ ]:
sorted(lem)

In [ ]:
m_dict['20050908_182943_22_fsp-A-1']['en_w']

In [ ]:
cat_speech_path = os.path.join(out_path, key)
wavs_path = os.path.join(out_path, "wavs")

In [ ]:
print("-"*80)
print("EPOCH = {0:d}".format(last_epoch+1))
fsh_pred_sents, fsh_utts, loss = feed_model(map_dict[key],
                  b_dict=bucket_dict[key],
                  vocab_dict=vocab_dict,
                  batch_size=BATCH_SIZE,
                  x_key=enc_key,
                  y_key=dec_key,
                  train=train,
                  cat_speech_path=cat_speech_path, use_y=True)

print("{0:s} {1:s} mean loss={2:.4f}".format("*" * 10,
                                    "train" if train else "dev",
                                    loss))
print("-")
print("-"*80)

In [ ]:
# Eval parameters
ref_index = -1
min_len, max_len= 0, 200
displayN = 50

In [ ]:
display_words(m_dict, v_dict, fsh_pred_sents[:displayN], fsh_utts[:displayN], 
              dec_key, key, min_len=min_len, max_len=max_len, play_audio=False)

In [ ]:
b, _, h, r = calc_bleu(m_dict, 
                              v_dict, 
                              fsh_pred_sents, 
                              fsh_utts, 
                              dec_key, 
                              min_len=min_len, 
                              max_len=max_len, 
                              ref_index=ref_index)

print("BLEU score on: {0:s} = {1:.2f}".format(key, b * 100))
print("-"*60)

# chrf = [0]*4

# for ref_i in range(4):
#     _, chrf[ref_i], _, _ = calc_bleu(m_dict, 
#                               v_dict, 
#                               fsh_pred_sents[:n], 
#                               fsh_utts[:n], 
#                               dec_key, 
#                               min_len=min_len, 
#                               max_len=max_len, 
#                               ref_index=ref_i)
#     # end for

# # print chrf score
# for i in range(4):
#     print("CHRF score on: {0:s} = {1:.2f}, using reference = {2:d}".format(key, chrf[i] * 100, i))

# print("-"*60)

all_weights=[(1.,0.,0.,0.),
             (0.,1.,0.,0.),
             (0.,0.,1.,0.),
             (0.,0.,0.,1.),
             (1./2,1./2,0.,0.),
             (1./3,1./3,1./3,0.),
             (.25,.25,.25,.25)]

In [ ]:
smooth_fun = nltk.translate.bleu_score.SmoothingFunction()

In [ ]:
print("{0:>20s} | {1:20s}".format("bleu score (0-100)", "uni-bi-tri-quad"))
for weights in all_weights:
    b = corpus_bleu(r, h, weights=weights, smoothing_function=smooth_fun.method2)
    print("{0:20.2f} | {1:20s}".format(b * 100, "-".join(map("{0:.2f}".format, weights))))

In [ ]:
_, _ = corpus_precision_recall(r, h)

In [ ]:
pred_path = "/afs/inf.ed.ac.uk/group/project/lowres/work/installs/fisher-callhome-corpus/corpus/ldc"
pred_fname = os.path.join(pred_path, "fisher_dev.pred")

In [ ]:
m_dict[fsh_utts[2]]['en_w']

In [ ]:
en_ref, en_hyp = write_predictions_to_file(m_dict, v_dict, fsh_pred_sents, fsh_utts, dec_key, key, min_len=min_len, max_len=max_len)

In [ ]:
len(en_ref),len(en_hyp)

### Callhome dev

In [ ]:
key='callhome_devtest'
train=False
m_dict = map_dict[key]
v_dict = vocab_dict[dec_key]
n=len(map_dict[key])
BATCH_SIZE = 100

In [ ]:
cat_speech_path = os.path.join(out_path, key)

In [ ]:
print("-"*80)
print("EPOCH = {0:d}".format(last_epoch+1))
call_pred_sents, call_utts, loss = feed_model(map_dict[key],
                  b_dict=bucket_dict[key],
                  vocab_dict=vocab_dict,
                  batch_size=BATCH_SIZE,
                  x_key=enc_key,
                  y_key=dec_key,
                  train=train,
                  cat_speech_path=cat_speech_path, use_y=True)

print("{0:s} {1:s} mean loss={2:.4f}".format("*" * 10,
                                    "train" if train else "dev",
                                    loss))
print("-")
print("-"*80)

In [ ]:
display_words(m_dict, v_dict, call_pred_sents[:displayN], call_utts[:displayN],
              dec_key, key, min_len=min_len, max_len=max_len, play_audio=False)

In [ ]:
b, _, h, r = calc_bleu(m_dict, 
                              v_dict, 
                              call_pred_sents, 
                              call_utts, 
                              dec_key, 
                              min_len=min_len, 
                              max_len=max_len, 
                              ref_index=ref_index)

print("BLEU score on: {0:s} = {1:.2f}".format(key, b * 100))
print("-"*60)

all_weights=[(1.,0.,0.,0.),
             (0.,1.,0.,0.),
             (0.,0.,1.,0.),
             (0.,0.,0.,1.),
             (1./2,1./2,0.,0.),
             (1./3,1./3,1./3,0.),
             (.25,.25,.25,.25)]

In [ ]:
smooth_fun = nltk.translate.bleu_score.SmoothingFunction()

In [ ]:
print("{0:>20s} | {1:20s}".format("bleu score (0-100)", "uni-bi-tri-quad"))
for weights in all_weights:
    b = corpus_bleu(r, h, weights=weights, smoothing_function=smooth_fun.method2)
    print("{0:20.2f} | {1:20s}".format(b * 100, "-".join(map("{0:.2f}".format, weights))))

In [ ]:
_, _ = corpus_precision_recall(r, h)

In [ ]:
call_en_ref, call_en_hyp = write_predictions_to_file(m_dict, v_dict, call_pred_sents, call_utts, 
                                                     dec_key, key, min_len=min_len, max_len=max_len)

In [ ]:
len(call_en_ref),len(call_en_hyp)

### Test code

In [ ]:
# test_ref = [["ha ha lol hue".split()], ["ha ha ja ha".split()], ["ha ha ja ha".split()]]
# test_h = ["lol ja".split(), "ha he".split(), "ha ja".split()]
# _, _ = corpus_precision_recall(test_ref, test_h)